In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('Interest_Rate.csv')
df.head(), df.tail(), len(df)

(       Date  4 WEEKS BANK DISCOUNT
 0  1/4/2021                   0.09
 1  1/5/2021                   0.08
 2  1/6/2021                   0.09
 3  1/7/2021                   0.09
 4  1/8/2021                   0.08,
            Date  4 WEEKS BANK DISCOUNT
 484   12/8/2022                   3.61
 485   12/9/2022                   3.66
 486  12/12/2022                   3.68
 487  12/13/2022                   3.71
 488  12/14/2022                   3.73,
 489)

In [6]:
### We assume 4-week riskfree rates p.a. in 2021 approx the riskfree short-rate p.a.
DY=df['4 WEEKS BANK DISCOUNT']  ### DY is annualized discount yield
### convert this DY to annualized spot rate SR (BEY)
###   discounted bill price P = 1-DY/12. SR approx by BEY = (1/p - 1) x 12 x 365/360. Hence.
SR=365*DY/(360*(1-DY/12))
print(SR)

0      0.091940
1      0.081655
2      0.091940
3      0.091940
4      0.081655
         ...   
484    5.235002
485    5.339329
486    5.381410
487    5.444914
488    5.487505
Name: 4 WEEKS BANK DISCOUNT, Length: 489, dtype: float64


In [7]:
DSR=SR.diff()
DSR=DSR[1:489,]
print(DSR)

1     -0.010284
2      0.010284
3      0.000000
4     -0.010284
5      0.010284
         ...   
484    0.020716
485    0.104327
486    0.042082
487    0.063503
488    0.042591
Name: 4 WEEKS BANK DISCOUNT, Length: 488, dtype: float64


In [8]:
SR1=SR.shift(1)
SR1=SR1[1:489,]
print(SR1)

1      0.091940
2      0.081655
3      0.091940
4      0.091940
5      0.081655
         ...   
484    5.214286
485    5.235002
486    5.339329
487    5.381410
488    5.444914
Name: 4 WEEKS BANK DISCOUNT, Length: 488, dtype: float64


In [35]:
from statsmodels.sandbox.regression.gmm import GMM

inst = np.column_stack((np.ones(len(SR1)), SR1))

class gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3 = params
        endog = self.endog
        exog = self.exog.squeeze()
        inst = self.instrument   

        error1 = endog - p0 - p1 * exog
        error2 = (endog - p0 - p1 * exog) ** 2 - p2 * (1/365) * (exog ** p3)
        error3 = (endog - p0 - p1 * exog) * inst[:,1]
        error4 = ((endog - p0 - p1 * exog) ** 2 - p2 * (1/365) * (exog ** p3)) * inst[:,1]
        g = np.column_stack((error1, error2, error3, error4))
        return g

initial0 = np.array([0.1, 0.1, 1, 1])
results = gmm(endog = DSR, exog = SR1, instrument = inst, k_moms=4, k_params=4).fit(initial0)

### numpy.squeeze() function is used when we want to remove single-dimensional entries from the shape of an array, e.g. [[[2,2],[3,3]]] to [[2,2],[3,3]]

Optimization terminated successfully.
         Current function value: 0.000002
         Iterations: 13
         Function evaluations: 17
         Gradient evaluations: 17
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 21
         Function evaluations: 27
         Gradient evaluations: 27
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1


In [15]:
print(results.summary())

                                   gmm Results                                   
Dep. Variable:     4 WEEKS BANK DISCOUNT   Hansen J:                    3.842e-11
Model:                               gmm   Prob (Hansen J):                   nan
Method:                              GMM                                         
Date:                   Thu, 22 Feb 2024                                         
Time:                           16:19:53                                         
No. Observations:                    488                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0            0.0052      0.002      2.311      0.021       0.001       0.010
p 1            0.0060      0.004      1.479      0.139      -0.002       0.014
p 2            3.4160      0.555      6.155      0.000       2.328       4.504
p 3            0.7662      0.10

In [16]:
mod = gmm(endog = DSR, exog = SR1, instrument = inst, k_moms=4, k_params=4)
mod.momcond(initial0).shape

(488, 4)

In [8]:
alpha=0.0052/(1/365); print("alpha:", alpha)
beta=-0.0060/(1/365); print("beta:", beta)  ### Note this is weakest - not significantly diff from zero. More like Dothan's model
sigma=np.sqrt(3.4160); print("sigma:", sigma)
lambda1 = 0.7662/2; print("lambda:", lambda1)

alpha: 1.898
beta: -2.19
sigma: 1.8482424083436675
lambda: 0.3831


# Suppose we use 5 moment conditions

In [9]:
from statsmodels.sandbox.regression.gmm import GMM

inst = np.column_stack((np.ones(len(SR1)), SR1))

class gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3 = params
        endog = self.endog
        exog = self.exog.squeeze()
        inst = self.instrument   

        error1 = endog - p0 - p1 * exog
        error2 = (endog - p0 - p1 * exog) ** 2 - p2 * (1/365) * (exog ** p3)
        error3 = (endog - p0 - p1 * exog) * inst[:,1]
        error4 = ((endog - p0 - p1 * exog) ** 2 - p2 * (1/365) * (exog ** p3)) * inst[:,1]
        error5 = (endog - p0 - p1 * exog) * (inst[:,1])**2
        g = np.column_stack((error1, error2, error3, error4, error5))
        return g

initial0 = np.array([0.1, 0.1, 1, 1])
results2 = gmm(endog = DSR, exog = SR1, instrument = inst, k_moms=5, k_params=4).fit(initial0)


Optimization terminated successfully.
         Current function value: 0.000039
         Iterations: 13
         Function evaluations: 19
         Gradient evaluations: 19
Optimization terminated successfully.
         Current function value: 0.009094
         Iterations: 23
         Function evaluations: 29
         Gradient evaluations: 29
Optimization terminated successfully.
         Current function value: 0.008856
         Iterations: 11
         Function evaluations: 16
         Gradient evaluations: 16
Optimization terminated successfully.
         Current function value: 0.008837
         Iterations: 14
         Function evaluations: 17
         Gradient evaluations: 17
Optimization terminated successfully.
         Current function value: 0.008836
         Iterations: 3
         Function evaluations: 5
         Gradient evaluations: 5
Optimization terminated successfully.
         Current function value: 0.008835
         Iterations: 6
         Function evaluations: 10
      

In [10]:
print(results2.summary())

                                   gmm Results                                   
Dep. Variable:     4 WEEKS BANK DISCOUNT   Hansen J:                        4.312
Model:                               gmm   Prob (Hansen J):                0.0379
Method:                              GMM                                         
Date:                   Mon, 20 Feb 2023                                         
Time:                           21:45:42                                         
No. Observations:                    488                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0            0.0019      0.002      1.183      0.237      -0.001       0.005
p 1            0.0065      0.004      1.604      0.109      -0.001       0.014
p 2            2.6468      0.476      5.557      0.000       1.713       3.580
p 3            0.8516      0.14

In [11]:
### Hansen J stats shows 4.312 with p-value 0.0379. Hence reject model at 5% significance level, but not at 1% significance level.

In [12]:
import scipy.stats

#find Chi-Square critical value
scipy.stats.chi2.ppf(1-.03785, df=1)

4.3117502675445145